In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [ ]:
#convert stata data to pandas
stata_data='../NSDUH_2017.DTA'
df=pd.read_stata(stata_data)

In [ ]:
df.shape[0]

In [ ]:
dfa = df[df['CATAG2']!='1 - 12-17 Years Old']
dfa = dfa[df.suicplan != '85 - BAD DATA Logically assigned']
dfa = dfa[df.suicplan != '94 - DON T KNOW']
dfa = dfa[df.suicplan != '97 - REFUSED']
dfa = dfa[df.suicplan != '98 - BLANK']

In [ ]:
dfa.shape[0]

In [ ]:
dfa.groupby(['suicplan']).size()

In [ ]:
dfa.loc[df.suicplan=='1 - Yes', 'suicide'] = 'plan'
dfa.loc[df.suicplan=='2 - No', 'suicide'] = 'thought'
dfa.loc[df.suicplan=='99 - LEGITIMATE SKIP', 'suicide'] = 'no'

In [ ]:
dfa.groupby('irpnrnmrec').size()

In [ ]:
dfa.loc[dfa.irpnrnmrec=='1 - Within the past 30 days', 'misuse'] = 'last year'
dfa.loc[dfa.irpnrnmrec=='2 - More than 30 days ago but within the past 12 mos', 'misuse'] = 'last year'
dfa.loc[dfa.irpnrnmrec=='3 - More than 12 months ago', 'misuse'] = 'sometime'
dfa.loc[dfa.irpnrnmrec=='9 - NEVER MISUSED PAIN RELIEVERS', 'misuse'] = 'never'

In [ ]:
dfb=dfa[dfa.misuse!='sometime']
dfb.shape[0]

In [ ]:
gf=dfb.groupby(['suicide','misuse']).size()

In [ ]:
#y=dfb.suicide
#dummies = pd.get_dummies(y)
#y=dummies['plan']

In [ ]:
#X=dfb.misuse
#X = 1-pd.get_dummies(X)['never']

In [ ]:
dfb['s_yes_no']=dfb['suicide'].apply(lambda x: 1 if x=='plan' else 0)
dfb['o_yes_no']=dfb['misuse'].apply(lambda x: 0 if x=='never' else 1)

In [ ]:
dfb[['o_yes_no','s_yes_no']].corr()

In [ ]:
X=dfb['o_yes_no']
X=np.array(X).reshape(-1,1)
y=dfb['s_yes_no']

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs',
...                          multi_class='multinomial').fit(X, y)

In [ ]:
coef=clf.coef_
coef

In [ ]:
np.exp(coef)

In [ ]:
dfb[dfb['s_yes_no']==1].shape[0]

In [ ]:
#not user suicide rate
dfb[(dfb['s_yes_no']==1) & (dfb['o_yes_no']==0)].shape[0]/dfb[dfb['o_yes_no']==0].shape[0]

In [ ]:
#users suicide rate
dfb[(dfb['s_yes_no']==1) & (dfb['o_yes_no']==1)].shape[0]/dfb[dfb['o_yes_no']==1].shape[0]